In [2]:
import sqlite3
conn = sqlite3.connect('nominations.db')
cursor = conn.cursor()

schema = cursor.execute('pragma table_info(nominations)').fetchall()
for row in schema :
    print(row)

(0, 'id', 'integer', 0, None, 1)
(1, 'category', 'text', 0, None, 0)
(2, 'nominee', 'text', 0, None, 0)
(3, 'movie', 'text', 0, None, 0)
(4, 'character', 'text', 0, None, 0)
(5, 'won', 'integer', 0, None, 0)
(6, 'ceremony_id', 'integer', 0, None, 0)


In [3]:
first_five = cursor.execute('select * from nominations limit 5').fetchall()
for row in first_five :
    print(row)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)


In [4]:
# Create the ceremonies table with the following schema

q_createCeremonies = '''
create table ceremonies (
    id integer primary key,
    Year integer,
    Host text
)
'''
cursor.execute(q_createCeremonies)

In [11]:
schema = cursor.execute('pragma table_info(ceremonies)').fetchall()
for row in schema :
    print(row)
    
ceremonie_data = cursor.execute('select * from ceremonies').fetchall()
for row in ceremonie_data :
    print(row)

(0, 'id', 'integer', 0, None, 1)
(1, 'Year', 'integer', 0, None, 0)
(2, 'Host', 'text', 0, None, 0)


In [12]:
# the Connection method executemany to insert the values into the ceremonies table

years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal")]
insert_query = 'insert into ceremonies (Year, Host) values (?, ?)'
cursor.executemany(insert_query, years_hosts)

In [13]:
ceremonie_data = cursor.execute('select * from ceremonies').fetchall()
for row in ceremonie_data :
    print(row)

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')
(11, 2000, 'Billy Crystal')


In [14]:
# Turn on foreign key constraints
conn.execute('PRAGMA foreign_keys = ON;')

In [18]:
# create a new table nominations_two with the schema we want

q_newNominations = '''
create table nominations_two (
    id integer primary key,
    category text,
    nominee text,
    movie text,
    character text,
    won integer,
    ceremony_id integer,
    foreign key (ceremony_id) references ceremonies(id)
)
'''
cursor.execute(q_newNominations)

In [20]:
cursor.execute('pragma table_info(nominations_two)').fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'character', 'text', 0, None, 0),
 (5, 'won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [27]:
# populate nominations_two with the records we want

q_populateData = '''
select nominations.category, nominations.nominee, nominations.movie, 
       nominations.character, nominations.won, ceremonies.id
from nominations
inner join ceremonies
on nominations.year == ceremonies.year
'''

joined_nominations = cursor.execute(q_populateData).fetchall()

q_insertNewNominations = '''
insert into nominations_two
(category, nominee, movie, character, won, ceremony_id)
values (?, ?, ?, ?, ?, ?)
'''
cursor.executemany(q_insertNewNominations, joined_nominations)

In [29]:
# Verify your work by returning the first 5 rows from nominations_two

ceremonie_data = cursor.execute('select * from nominations_two limit 5').fetchall()
for row in ceremonie_data :
    print(row)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)


In [30]:
# Deleting and renaming tables
# delete the original nominations table, rename nominations_two to nominations

conn.execute('drop table nominations')
conn.execute('alter table nominations_two rename to nominations')

In [33]:
# Create the 3 tables we need to model the relationship between movies and actors.

q_movies = '''
create table movies (
    id integer primary key,
    movie text
)
'''
q_actors = '''
create table actors (
    id integer primary key,
    actor text
)
'''
q_moviesActors  = '''
create table movies_actors (
    id integer primary key,
    movie_id integer references movies(id),
    actor_id integer references actors(id)
)
'''

conn.execute(q_movies)
conn.execute(q_actors)
conn.execute(q_moviesActors)

In [42]:
# The SQL INSERT INTO SELECT Statement
# https://www.w3schools.com/sql/sql_insert_into_select.asp

q_insertMovies = '''
insert into movies (movie)
select distinct(movie) from nominations
'''
q_insertActors = '''
insert into actors (actor)
select distinct(nominee) from nominations
'''

cursor.execute(q_insertMovies)
cursor.execute(q_insertActors)

In [43]:
# Verify your work by returning the first 5 rows from movies & actors

movies_data = cursor.execute('select * from movies limit 5').fetchall()
for row in movies_data :
    print(row)

actors_data = cursor.execute('select * from actors limit 5').fetchall()
for row in actors_data :
    print(row)

(1, 'Biutiful')
(2, 'True Grit')
(3, 'The Social Network')
(4, "The King's Speech")
(5, '127 Hours')
(1, 'Javier Bardem')
(2, 'Jeff Bridges')
(3, 'Jesse Eisenberg')
(4, 'Colin Firth')
(5, 'James Franco')


In [46]:
# populate the join table and the linked tables using data from nominations

q_nominations = "select movie, nominee from nominations"
movie_nominee = cursor.execute(q_nominations).fetchall()
# movie_nominee

q_insertMoviesActors = '''
insert into movies_actors (movie_id, actor_id)
values ( (select id from movies where movie == ?),
         (select id from actors where actor == ?) )
'''
cursor.executemany(q_insertMoviesActors, movie_nominee)

In [47]:
# Verify your work by returning the first 10 rows from movies_actors

movies_actors_data = cursor.execute('select * from movies_actors limit 10').fetchall()
for row in movies_actors_data :
    print(row)

(1, 1, 1)
(2, 2, 2)
(3, 3, 3)
(4, 4, 4)
(5, 5, 5)
(6, 6, 6)
(7, 7, 7)
(8, 8, 8)
(9, 9, 9)
(10, 4, 10)
